In [1]:
import pandas as pd
import sqlite3
import json, csv
import geopandas
import re
import numpy as np
import time

In [2]:
address_points = pd.read_csv('cook_locations.csv')
address_points.head()

/tmp/ipykernel_9453/3716568878.py:1: DtypeWarning: Columns (1,5,7,11,17,25,26,27,30,31,32,36,38,40,46,47,48,52,54) have mixed types. Specify dtype option on import or set low_memory=False.
  address_points = pd.read_csv('cook_locations.csv')


,OBJECTID,AddNum_Pre,Add_Number,AddNum_Suf,ADDRNOCOM,St_PreMod,St_PreDir,St_PreTyp,St_Name,St_PosTyp,...,Seat,Inc_Muni,Uninc_Comm,MSAGComm,DiscrpAgID,Site_NGUID,ESN,TWP_NAME,LSt_PreDir,LSt_Type
0,1,NaN,1763.0,NaN,1763,NaN,WEST,NaN,WELLINGTON,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,1622.0,NaN,1622,NaN,WEST,NaN,CULLERTON,STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,917.0,NaN,917,NaN,WEST,NaN,ARMITAGE,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,20018.0,NaN,20018,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,20024.0,NaN,20024,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
address_points.columns

Index(['OBJECTID', 'AddNum_Pre', 'Add_Number', 'AddNum_Suf', 'ADDRNOCOM',
       'St_PreMod', 'St_PreDir', 'St_PreTyp', 'St_Name', 'St_PosTyp',
       'St_PosDir', 'St_PosMod', 'STNAMECOM', 'LandmkName', 'PLACENAME',
       'Post_Comm', 'State', 'Post_Code', 'Post_Code4', 'ADDRDELIV',
       'CMPADDABRV', 'Long', 'Lat', 'USGRIDCORD', 'Elev', 'PIN', 'Effective',
       'Expire', 'EDITTIME', 'geocode_muni', 'Document_Name', 'created_user',
       'created_date', 'last_edited_user', 'last_edited_date', 'Source',
       'Source_Comment', 'Structure', 'OrthoYear', 'Placement', 'County',
       'Building', 'Floor', 'Unit', 'Room', 'Seat', 'Inc_Muni', 'Uninc_Comm',
       'MSAGComm', 'DiscrpAgID', 'Site_NGUID', 'ESN', 'TWP_NAME', 'LSt_PreDir',
       'LSt_Type'],
      dtype='object')

In [4]:
cook_locations = address_points[['Add_Number', 'ADDRDELIV','CMPADDABRV','Lat', 'Long','PLACENAME','Post_Code']]
cook_locations.head()

,Add_Number,ADDRDELIV,CMPADDABRV,Lat,Long,PLACENAME,Post_Code
0,1763.0,1763 WEST WELLINGTON AVENUE,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657
1,1622.0,1622 WEST CULLERTON STREET,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608
2,917.0,917 WEST ARMITAGE AVENUE,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614
3,20018.0,20018 MOHAWK TRAIL,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461
4,20024.0,20024 MOHAWK TRAIL,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461


In [5]:
cook_locations.dtypes

Add_Number    float64
ADDRDELIV      object
CMPADDABRV     object
Lat           float64
Long          float64
PLACENAME      object
Post_Code      object
dtype: object

In [6]:
cook_locations.shape

(1347092, 7)

In [8]:
#get rid of nan values

In [7]:
cook_locations_clean = cook_locations[~cook_locations['Post_Code'].isnull()]
cook_locations_clean

,Add_Number,ADDRDELIV,CMPADDABRV,Lat,Long,PLACENAME,Post_Code
0,1763.0,1763 WEST WELLINGTON AVENUE,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657
1,1622.0,1622 WEST CULLERTON STREET,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608
2,917.0,917 WEST ARMITAGE AVENUE,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614
3,20018.0,20018 MOHAWK TRAIL,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461
4,20024.0,20024 MOHAWK TRAIL,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461
...,...,...,...,...,...,...,...
1347087,222.0,222 MERCHANDISE MART PLAZA,222 MERCHANDISE MART PLZ,NaN,NaN,CHICAGO,60654
1347088,1410.0,1410 SOUTH MUSEUM CAMPUS DRIVE,1410 S MUSEUM CAMPUS DR,NaN,NaN,CHICAGO,60605
1347089,NaN,6660 WEST 66TH STREET,6660 W 66TH ST,NaN,NaN,BEDFORD PARK,60638
1347090,9590.0,9590 WEST 55TH STREET,9590 W 55TH ST,NaN,NaN,COUNTRYSIDE,60525


In [9]:
#get rid of empty strings
#cast as a string
#check the success rate if it about 90%


In [10]:
#cook_locations_clean[cook_locations_clean['Post_Code'] == ' ']

In [15]:
#cook_locations_clean = cook_locations_clean[cook_locations_clean['Post_Code'] != ' ']
#cook_locations_clean

,Add_Number,ADDRDELIV,CMPADDABRV,Lat,Long,PLACENAME,Post_Code
0,1763.0,1763 WEST WELLINGTON AVENUE,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657
1,1622.0,1622 WEST CULLERTON STREET,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608
2,917.0,917 WEST ARMITAGE AVENUE,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614
3,20018.0,20018 MOHAWK TRAIL,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461
4,20024.0,20024 MOHAWK TRAIL,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461
...,...,...,...,...,...,...,...
1347087,222.0,222 MERCHANDISE MART PLAZA,222 MERCHANDISE MART PLZ,NaN,NaN,CHICAGO,60654
1347088,1410.0,1410 SOUTH MUSEUM CAMPUS DRIVE,1410 S MUSEUM CAMPUS DR,NaN,NaN,CHICAGO,60605
1347089,NaN,6660 WEST 66TH STREET,6660 W 66TH ST,NaN,NaN,BEDFORD PARK,60638
1347090,9590.0,9590 WEST 55TH STREET,9590 W 55TH ST,NaN,NaN,COUNTRYSIDE,60525


In [8]:
cook_locations_clean.dtypes

Add_Number    float64
ADDRDELIV      object
CMPADDABRV     object
Lat           float64
Long          float64
PLACENAME      object
Post_Code      object
dtype: object

In [18]:
#cook_locations_clean['Post_Code'] = cook_locations_clean['Post_Code'].astype(str).astype(int)

/tmp/ipykernel_441706/3008046163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cook_locations_clean['Post_Code'] = cook_locations_clean['Post_Code'].astype(str).astype(int)


In [9]:
omec_locations = pd.read_csv('oemc_locations.csv')
omec_locations.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD


In [10]:
omec_locations.shape

(171266, 12)

In [11]:
omec_locations.dtypes

EventNumber          int64
EntryDate           object
EventType           object
TypeDescription     object
FinalDisposition    object
Location            object
CPDUnitList         object
CFDUnitList         object
numerical           object
directional         object
street_name         object
suffix              object
dtype: object

In [12]:
#check for alphabets in numerical column

In [13]:
omec_locations[omec_locations['numerical'].str.isalpha()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
1250,2000801443,01/08/2020 03.15.54,THREAS,THREATENING SUICIDE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
33053,2017807536,06/26/2020 13.27.24,ASLTIP,ASSAULT IP,FILE,RIDGE & AUSTIN,NaN,NaN,RIDGE,&,NaN,AUSTIN
34323,2018417148,07/02/2020 22.13.10,ATSUCD,ATTEMPT SUICIDE,7I,TRUMP INTERNATIONAL HOTE,1834 1832,A42,TRUMP,INTERNATIONAL,NaN,HOTE
40557,2021610499,08/03/2020 17.14.11,NPS,NO POLICE SERVICE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
67246,2035401094,12/19/2020 01.30.42,THREAS,THREATENING SUICIDE,FILE,CLAYTON MOTEL/ SNOW AVENUE,LOVES PARK,IL,CLAYTON,MOTEL/,SNOW,AVENUE
77674,2104513267,02/14/2021 22.17.20,THREAS,THREATENING SUICIDE,FILE,NEW ENGLAND/ X TH,NaN,NaN,NEW,ENGLAND/,X,TH
98438,2114909460,05/29/2021 14.13.31,DISTME,MENTAL HEALTH DISTURBANCE,FILE,W 87TH ST,NaN,NaN,W,87TH,NaN,ST
108404,2119503444,07/14/2021 08.44.18,DISTME,MENTAL HEALTH DISTURBANCE,19P,ILLINOIS MCCLURG CT,1802S 1823 1834,NaN,ILLINOIS,MCCLURG,NaN,CT
109196,2119812628,07/17/2021 18.51.00,MISSPE,MISSING PERSON,NaN,TRAINING EXERCISE ONLY,NaN,NaN,TRAINING,EXERCISE,NaN,ONLY
113218,2121707009,08/05/2021 12.48.58,DEATHR,DEATH REMOVAL,NaN,SWEDISH COVENANT HOSPITAL,1701,NaN,SWEDISH,COVENANT,NaN,HOSPITAL


In [14]:
omec_locations[omec_locations['numerical'].str.isupper()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
1250,2000801443,01/08/2020 03.15.54,THREAS,THREATENING SUICIDE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
33053,2017807536,06/26/2020 13.27.24,ASLTIP,ASSAULT IP,FILE,RIDGE & AUSTIN,NaN,NaN,RIDGE,&,NaN,AUSTIN
34323,2018417148,07/02/2020 22.13.10,ATSUCD,ATTEMPT SUICIDE,7I,TRUMP INTERNATIONAL HOTE,1834 1832,A42,TRUMP,INTERNATIONAL,NaN,HOTE
36638,2019615318,07/14/2020 21.29.09,SUSAUW,SUSPICIOUS AUTO WITH OCC,19PZ,N1198 N CICERO AV /48XX W DIVISION ST,1533,NaN,N1198,N,CICERO AV /48XX W DIVISION,ST
40557,2021610499,08/03/2020 17.14.11,NPS,NO POLICE SERVICE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
41322,2022004972,08/07/2020 10.44.10,CHILDP,CHILD PLAYING ON PHONE,FILE,A655 W 65TH ST,NaN,NaN,A655,W,65TH,ST
67246,2035401094,12/19/2020 01.30.42,THREAS,THREATENING SUICIDE,FILE,CLAYTON MOTEL/ SNOW AVENUE,LOVES PARK,IL,CLAYTON,MOTEL/,SNOW,AVENUE
77674,2104513267,02/14/2021 22.17.20,THREAS,THREATENING SUICIDE,FILE,NEW ENGLAND/ X TH,NaN,NaN,NEW,ENGLAND/,X,TH
97041,2114221111,05/22/2021 23.41.27,THREAS,THREATENING SUICIDE,FILE,AS5700-57XX W SCHOOL ST,NaN,NaN,AS5700-5700,W,SCHOOL,ST
98438,2114909460,05/29/2021 14.13.31,DISTME,MENTAL HEALTH DISTURBANCE,FILE,W 87TH ST,NaN,NaN,W,87TH,NaN,ST


In [15]:
omec_clean_locations = omec_locations[~omec_locations['numerical'].str.isupper()]

In [16]:
omec_clean_locations.shape

(171248, 12)

In [17]:
omec_clean_locations[~omec_clean_locations['numerical'].str.isnumeric()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
21419,2011910398,04/28/2020 17.56.09,DIST,DISTURBANCE,5FZ,\2844 N BROADWAY,1935,NaN,\2844,N,NaN,BROADWAY


21419    \2844 N BROADWAY
Name: Location, dtype: object

In [ ]:
#.str.strip
#check for

In [18]:
omec_clean_locations = omec_clean_locations[omec_clean_locations['EventNumber'] != 2011910398 ]

In [19]:
omec_clean_locations.shape

(171247, 12)

In [20]:
omec_clean_locations.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD


In [64]:
#omec_clean_locations['numerical'] = omec_clean_locations['numerical'].astype(str).astype(int)

In [21]:
omec_clean_locations['numerical'] = omec_clean_locations['numerical'].astype(str)

In [22]:
omec_clean_locations['numerical'] = omec_clean_locations['numerical'].str.strip()

In [23]:
omec_clean_locations['numerical'] = omec_clean_locations['numerical'].astype(int)

In [24]:
omec_clean_locations.dtypes

EventNumber          int64
EntryDate           object
EventType           object
TypeDescription     object
FinalDisposition    object
Location            object
CPDUnitList         object
CFDUnitList         object
numerical            int64
directional         object
street_name         object
suffix              object
dtype: object

In [25]:
#strip the any trailing white space
omec_clean_locations['directional'] = omec_clean_locations['directional'].str.strip()

In [26]:
omec_clean_locations.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD


In [27]:
def fullDirections(direction):
    if direction == 'W':
        return 'WEST'
    if direction == 'E':
        return 'EAST'
    if direction == 'S':
        return 'SOUTH'
    if direction == 'N':
        return 'NORTH'
    else:
        return direction
    

In [28]:
omec_clean_locations['St_PreDir']= omec_clean_locations['directional'].apply(fullDirections)
omec_clean_locations.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,SOUTH
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST


In [29]:
omec_clean_locations[omec_clean_locations['street_name'] == 'DR MARTIN LUTHER KING JR']['suffix'] = ''

/tmp/ipykernel_9453/1182088338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omec_clean_locations[omec_clean_locations['street_name'] == 'DR MARTIN LUTHER KING JR']['suffix'] = ''


In [30]:
omec_clean_locations.to_csv('cleaned_oemc_locations.csv')

In [72]:
#check out cook county address data and subset to columns needed for geocoding

In [31]:
address_points.head()

,OBJECTID,AddNum_Pre,Add_Number,AddNum_Suf,ADDRNOCOM,St_PreMod,St_PreDir,St_PreTyp,St_Name,St_PosTyp,...,Seat,Inc_Muni,Uninc_Comm,MSAGComm,DiscrpAgID,Site_NGUID,ESN,TWP_NAME,LSt_PreDir,LSt_Type
0,1,NaN,1763.0,NaN,1763,NaN,WEST,NaN,WELLINGTON,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,1622.0,NaN,1622,NaN,WEST,NaN,CULLERTON,STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,917.0,NaN,917,NaN,WEST,NaN,ARMITAGE,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,20018.0,NaN,20018,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,20024.0,NaN,20024,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
address_points.columns

Index(['OBJECTID', 'AddNum_Pre', 'Add_Number', 'AddNum_Suf', 'ADDRNOCOM',
       'St_PreMod', 'St_PreDir', 'St_PreTyp', 'St_Name', 'St_PosTyp',
       'St_PosDir', 'St_PosMod', 'STNAMECOM', 'LandmkName', 'PLACENAME',
       'Post_Comm', 'State', 'Post_Code', 'Post_Code4', 'ADDRDELIV',
       'CMPADDABRV', 'Long', 'Lat', 'USGRIDCORD', 'Elev', 'PIN', 'Effective',
       'Expire', 'EDITTIME', 'geocode_muni', 'Document_Name', 'created_user',
       'created_date', 'last_edited_user', 'last_edited_date', 'Source',
       'Source_Comment', 'Structure', 'OrthoYear', 'Placement', 'County',
       'Building', 'Floor', 'Unit', 'Room', 'Seat', 'Inc_Muni', 'Uninc_Comm',
       'MSAGComm', 'DiscrpAgID', 'Site_NGUID', 'ESN', 'TWP_NAME', 'LSt_PreDir',
       'LSt_Type'],
      dtype='object')

In [33]:
columns = ['CMPADDABRV', 'Lat', 'Long', 'PLACENAME', 'Post_Code', 'OBJECTID', 'Add_Number','St_PreDir', 'St_Name']

In [34]:
database = address_points[columns]
database 

,CMPADDABRV,Lat,Long,PLACENAME,Post_Code,OBJECTID,Add_Number,St_PreDir,St_Name
0,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657,1,1763.0,WEST,WELLINGTON
1,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608,2,1622.0,WEST,CULLERTON
2,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614,3,917.0,WEST,ARMITAGE
3,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461,4,20018.0,NaN,MOHAWK
4,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461,5,20024.0,NaN,MOHAWK
...,...,...,...,...,...,...,...,...,...
1347087,222 MERCHANDISE MART PLZ,NaN,NaN,CHICAGO,60654,1465915,222.0,NaN,MERCHANDISE MART
1347088,1410 S MUSEUM CAMPUS DR,NaN,NaN,CHICAGO,60605,1465916,1410.0,SOUTH,MUSEUM CAMPUS
1347089,6660 W 66TH ST,NaN,NaN,BEDFORD PARK,60638,1465917,NaN,WEST,66TH
1347090,9590 W 55TH ST,NaN,NaN,COUNTRYSIDE,60525,1465918,9590.0,WEST,55TH


In [35]:
database = database[database['PLACENAME'] =='Chicago']

In [36]:
database.shape

(582251, 9)

## Chunking the dataframes into mini Dataframes

In [37]:
#split array into chunks in order to run the process on my laptop

In [38]:
chunkDf = np.array_split(omec_clean_locations,30)
chunkDf

[      EventNumber            EntryDate EventType            TypeDescription  \
 0      2000100015  01/01/2020 00.00.52    DISTME  MENTAL HEALTH DISTURBANCE   
 1      2000100451  01/01/2020 00.21.18      DIST                DISTURBANCE   
 2      2000100656  01/01/2020 00.33.55    DOMBAT           DOMESTIC BATTERY   
 3      2000100660  01/01/2020 00.34.00    DISTME  MENTAL HEALTH DISTURBANCE   
 4      2000100902  01/01/2020 00.47.02    CHECWB           CHECK WELL BEING   
 ...           ...                  ...       ...                        ...   
 5705   2003312092  02/02/2020 17.59.38    DISTME  MENTAL HEALTH DISTURBANCE   
 5706   2003312108  02/02/2020 18.00.43    ASLTIP                 ASSAULT IP   
 5707   2003312147  02/02/2020 18.03.29      DIST                DISTURBANCE   
 5708   2003312242  02/02/2020 18.11.31    CHECWB           CHECK WELL BEING   
 5709   2003312295  02/02/2020 18.17.03    DISTME  MENTAL HEALTH DISTURBANCE   
 
      FinalDisposition                

In [41]:
#chunkDf = np.array_split(omec_clean_locations,600)
#chunkDf

[     EventNumber            EntryDate EventType            TypeDescription  \
 0     2000100015  01/01/2020 00.00.52    DISTME  MENTAL HEALTH DISTURBANCE   
 1     2000100451  01/01/2020 00.21.18      DIST                DISTURBANCE   
 2     2000100656  01/01/2020 00.33.55    DOMBAT           DOMESTIC BATTERY   
 3     2000100660  01/01/2020 00.34.00    DISTME  MENTAL HEALTH DISTURBANCE   
 4     2000100902  01/01/2020 00.47.02    CHECWB           CHECK WELL BEING   
 ..           ...                  ...       ...                        ...   
 281   2000209506  01/02/2020 15.48.30      DIST                DISTURBANCE   
 282   2000209508  01/02/2020 15.48.43     BATIP                 BATTERY IP   
 283   2000209536  01/02/2020 15.50.21      ASST               ASSIST OTHER   
 284   2000209545  01/02/2020 15.50.52    DISTME  MENTAL HEALTH DISTURBANCE   
 285   2000209618  01/02/2020 15.55.29    ASLTIP                 ASSAULT IP   
 
     FinalDisposition                Location  \
 

In [39]:
chunkDf[0]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,SOUTH
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,2003312092,02/02/2020 17.59.38,DISTME,MENTAL HEALTH DISTURBANCE,15I,11XX E 50TH ST,222,NaN,1100,E,50TH,ST,EAST
5706,2003312108,02/02/2020 18.00.43,ASLTIP,ASSAULT IP,5B,59XX S SAWYER AV,823,NaN,5900,S,SAWYER,AV,SOUTH
5707,2003312147,02/02/2020 18.03.29,DIST,DISTURBANCE,5PZ,15XX N CLYBOURN AV,1822,NaN,1500,N,CLYBOURN,AV,NORTH
5708,2003312242,02/02/2020 18.11.31,CHECWB,CHECK WELL BEING,19PZ,14XX W PRATT BL,2433 2432,NaN,1400,W,PRATT,BL,WEST


## Helper Functions

In [ ]:
#create lambda function and use .apply instead of iterrow

In [40]:
chunkHead = chunkDf[0].head()
chunkHead

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,SOUTH
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST


In [ ]:
#list_zipcodes{i} = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)

In [53]:
test_results = chunkHead.apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)

In [45]:
test_results

0    ([60620], [41.75590425, 41.75534803, 41.754398...
1    ([60655], [41.70491317, 41.70578963, 41.704799...
2    ([60641], [41.94070996, 41.94066717, 41.940666...
3                                         ([], [], [])
4    ([60634], [41.95379298, 41.95332232, 41.953322...
dtype: object

In [39]:
test_results

0    ([60620], [41.75579216, 41.75583129, 41.757156...
1    ([60655], [41.70491317, 41.70578963, 41.704799...
2    ([60641, 60618], [41.94071132, 41.94063773, 41...
3                                         ([], [], [])
4    ([60634], [41.95270475, 41.95269363, 41.953792...
dtype: object

In [68]:
geoCodeToDf(test_results,['zipcodes','lats','longs'])

In [58]:
loc = pd.DataFrame.from_records(test_results, columns=['zipcodes','lats','longs'])


In [55]:
loc

,zipcode,lats,longs
0,[60620],"[41.75590425, 41.75534803, 41.75439881, 41.755...","[-87.64385433, -87.64383799, -87.64450703, -87..."
1,[60655],"[41.70491317, 41.70578963, 41.70479904, 41.704...","[-87.70410419, -87.70413558, -87.70410952, -87..."
2,[60641],"[41.94070996, 41.94066717, 41.94066687, 41.940...","[-87.72948623, -87.72867574, -87.72931246, -87..."
3,[],[],[]
4,[60634],"[41.95379298, 41.95332232, 41.95332206, 41.953...","[-87.7900676, -87.79091283, -87.79039137, -87...."


In [59]:
test_results_final = pd.concat([chunkHead, loc], axis=1)
test_results_final

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir,zipcodes,lats,longs
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,SOUTH,[60620],"[41.75590425, 41.75534803, 41.75439881, 41.755...","[-87.64385433, -87.64383799, -87.64450703, -87..."
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH,[60655],"[41.70491317, 41.70578963, 41.70479904, 41.704...","[-87.70410419, -87.70413558, -87.70410952, -87..."
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST,[60641],"[41.94070996, 41.94066717, 41.94066687, 41.940...","[-87.72948623, -87.72867574, -87.72931246, -87..."
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5,[],[],[]
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST,[60634],"[41.95379298, 41.95332232, 41.95332206, 41.953...","[-87.7900676, -87.79091283, -87.79039137, -87...."


In [60]:
test_results_final['zipcode'] = test_results_final['zipcodes'].apply(extractZip)

In [61]:
test_results_final['lat'] = test_results_final['lats'].apply(extractZip)

In [62]:
test_results_final['long'] = test_results_final['longs'].apply(extractZip)

In [63]:
test_results_final

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir,zipcodes,lats,longs,zipcode,lat,long
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,SOUTH,[60620],"[41.75590425, 41.75534803, 41.75439881, 41.755...","[-87.64385433, -87.64383799, -87.64450703, -87...",60620.0,41.755904,-87.643854
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH,[60655],"[41.70491317, 41.70578963, 41.70479904, 41.704...","[-87.70410419, -87.70413558, -87.70410952, -87...",60655.0,41.704913,-87.704104
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST,[60641],"[41.94070996, 41.94066717, 41.94066687, 41.940...","[-87.72948623, -87.72867574, -87.72931246, -87...",60641.0,41.940710,-87.729486
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5,[],[],[],NaN,NaN,NaN
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST,[60634],"[41.95379298, 41.95332232, 41.95332206, 41.953...","[-87.7900676, -87.79091283, -87.79039137, -87....",60634.0,41.953793,-87.790068


In [59]:
len(test_results[0][1])

34

In [60]:
len(test_results[0][2])

34

In [61]:
len(test_results[0][0])

1

In [ ]:
chunkHead['zipcodes'] =test_results[0][0]

In [ ]:
chunkHead['lats'] = test_results[0][1]

In [ ]:
chunkHead['longs'] = test_results[0][2]

In [52]:
def geoCodeChunk(numerical,direction,street_name,database_df):
    
    complete_conditional = database_df[(database_df['St_Name'] == street_name) 
                                & (database_df['St_PreDir'] == direction)
                                & (database_df['Add_Number'] < numerical + 100)
                                & (database_df['Add_Number'] >= numerical)
                                ]
    
    '''
     Improve selection criteria from the result set
    '''
    #result = complete_conditional['Post_Code']
    #zipcode = result['Post_Code']
    #lat = result['Lat']
    #long = result['Long']
    
    list_zipcodes = list(set(complete_conditional['Post_Code']))
    list_lat = list(set(complete_conditional['Lat']))
    list_long = list(set(complete_conditional['Long']))
    
#     try:
#         zipcode = list_zipcodes[0]
#         lat = list_lat[0]
#         long = list_long[0]
        
    
    return list_zipcodes,list_lat ,list_long
    #return zipcode, lat,long
    #return result

In [69]:
def geoCodeToDf(chunkDfName,columnNamesList):
    #chunkDfName[newColumnName]= chunkZipCodeList
    newDataframe = pd.DataFrame.from_records(chunkDfName, columns=[columnNamesList])
    return newDataframe
    #chunkDfName[number].to_csv(f'geocoded_batches/batch{number}.csv')

In [57]:
def extractZip(zipCode):
    try:
        return zipCode[0]
    except:
        return None

In [ ]:
loc = pd.DataFrame.from_records(test_results, columns=['zipcode','lats','longs'])


def geoCodeChunk(numerical,direction,street_name,database_df):
    start_time = time.time()    
    complete_conditional = database_df[(database_df['St_Name'] == street_name) 
                                & (database_df['St_PreDir'] == direction)
                                & (database_df['Add_Number'] < numerical + 100)
                                & (database_df['Add_Number'] >= numerical -100)
                                ]
    list_zipcodes = list(set(complete_conditional['Post_Code']))
    #print('Done')
    #print("--- %s minutes ---" % ((time.time() - start_time)/60))
    return list_zipcodes

def geoCodeToDf(number,chunkDfName,chunkZipCodeList):
    chunkDfName[number]['zipcodes']= chunkZipCodeList
    #chunkDfName[number].to_csv(f'geocoded_batches/batch{number}.csv')

In [62]:
#creates 30 empty list 
#name convention list_zipcodes0, list_zipcodes1,..etc
for k in range(30):
    exec(f'list_results{k} = []')

In [63]:
oemc_output = None

In [64]:
oemc_output = pd.DataFrame(columns=['EventNumber', 'EntryDate', 'EventType', 'TypeDescription', 'FinalDisposition', 'Location', 'CPDUnitList', 'CFDUnitList', 'numerical', 'directional', 'street_name', 'suffix' , 'zipcodes','lats','longs'])

In [65]:
oemc_output

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs


In [67]:
list_results = chunkDf[0].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)


In [70]:
list_results

0      ([60620], [41.75579216, 41.75583129, 41.757156...
1      ([60655], [41.70491317, 41.70578963, 41.704799...
2      ([60641, 60618], [41.94071132, 41.94063773, 41...
3                                           ([], [], [])
4      ([60634], [41.95270475, 41.95269363, 41.953792...
                             ...                        
281    ([60647, 60647], [41.92517474, 41.92516705, 41...
282    ([60610], [41.90025714, 41.90119052, 41.899858...
283    ([60628], [41.69149437, 41.69029838, 41.691864...
284    ([60651, 60647], [41.91041601, 41.90986361, 41...
285    ([60610], [41.90497316, 41.9047316, 41.9047764...
Length: 286, dtype: object

In [72]:
chunkDfName[newColumnName]= chunkZipCodeList

NameError: name 'chunkZipCodeList' is not defined

In [73]:
chunkDf[0]['zipcodes'] = list_results[0][0]

ValueError: Length of values (1) does not match length of index (286)

In [ ]:
chunkDf[0].head()

In [ ]:
chunkDf[0].apply(geoCodeToDf())

In [ ]:
geoCodeToDf(0,chunkDf,"zipcodes",list_results[0][0])

In [66]:
for i in range(1):
    exec(f'list_results = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')
    exec(f'geoCodeToDf(i,chunkDf,"zipcodes",list_results[0][0])')
    #exec(f'geoCodeToDf(i,chunkDf,"lats",list_results{i}[0][1])')
    #exec(f'geoCodeToDf(i,chunkDf,"longs",list_results{i}[0][2])')
    #exec(f'oemc_output = pd.concat([oemc_output,chunkDf[{i}]])')
print('Done')

ValueError: Length of values (1) does not match length of index (286)

In [150]:
for i in range(6):
    exec(f'list_zipcodes{i} = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')
    exec(f'geoCodeToDf(i,chunkDf,list_zipcodes{i})')
    exec(f'oemc_output = pd.concat([oemc_output,chunkDf[{i}]])')
    

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


...

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [151]:
oemc_output.shape

(39963, 14)

In [154]:
for i in range(6,12):
    exec(f'list_zipcodes{i} = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')
    exec(f'geoCodeToDf(i,chunkDf,list_zipcodes{i})')
    exec(f'oemc_output = pd.concat([oemc_output,chunkDf[{i}]])')
print('Done')

Done


In [155]:
oemc_output.shape

(74212, 14)

In [ ]:
oemc_output.to_csv('oemc_geocoded_final_batch.csv')

In [157]:
for i in range(12,18):
    exec(f'list_zipcodes{i} = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')
    exec(f'geoCodeToDf(i,chunkDf,list_zipcodes{i})')
    exec(f'oemc_output = pd.concat([oemc_output,chunkDf[{i}]])')
print('Done')

Done


In [158]:
oemc_output.shape

(108460, 14)

In [159]:
for i in range(18,24):
    exec(f'list_zipcodes{i} = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')
    exec(f'geoCodeToDf(i,chunkDf,list_zipcodes{i})')
    exec(f'oemc_output = pd.concat([oemc_output,chunkDf[{i}]])')
print('Done')

Done


In [160]:
oemc_output.shape

(142708, 14)

In [161]:
for i in range(24,30):
    exec(f'list_zipcodes{i} = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')
    exec(f'geoCodeToDf(i,chunkDf,list_zipcodes{i})')
    exec(f'oemc_output = pd.concat([oemc_output,chunkDf[{i}]])')
print('Done')

Done


In [162]:
oemc_output.shape

(176956, 14)

In [163]:
oemc_output.to_csv('oemc_geocoded_final_batch.csv')

In [164]:
oemc_output.shape

(176956, 14)

In [165]:
oemc_output.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,St_PreDir
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,[60620],SOUTH
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,[60655],SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,"[60641, 60618]",WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,[],.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,[60634],WEST


In [ ]:
#Extract the first zip codes from the zipcodes column

In [170]:
def extractZip(zipCode):
    try:
        return zipCode[0]
    except:
        return None
    

In [171]:
oemc_output['cleanZipCodes'] = oemc_output['zipcodes'].apply(extractZip)

In [172]:
oemc_output.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,St_PreDir,cleanZipCodes
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,[60620],SOUTH,60620
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,[60655],SOUTH,60655
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,"[60641, 60618]",WEST,60641
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,[],.5,None
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,[60634],WEST,60634


In [173]:
oemc_output[oemc_output['cleanZipCodes'].isnull()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,St_PreDir,cleanZipCodes
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,[],.5,None
16,2000102204,01/01/2020 01.59.07,THREAS,THREATENING SUICIDE,19P,5XX N CLARK ST /1XX W GRAND AV,1860,NaN,500,N,CLARK ST /1XX W GRAND,AV,[],NORTH,None
26,2000103846,01/01/2020 03.28.07,DISTME,MENTAL HEALTH DISTURBANCE,15P,91XX S DR MARTIN LUTHER KING JR DR,632R,NaN,9100,S,DR MARTIN LUTHER KING JR,DR,[],SOUTH,None
28,2000104170,01/01/2020 03.47.33,THREAS,THREATENING SUICIDE,19B,7XX E 87TH ST /87XX S COTTAGE GROVE AV,631R,NaN,700,E,87TH ST /87XX S COTTAGE GROVE,AV,[],EAST,None
37,2000105293,01/01/2020 05.19.12,BATIP,BATTERY IP,19IZ,4XX E 63RD ST /62XX S VERNON AV,320R 312R 313R,NaN,400,E,63RD ST /62XX S VERNON,AV,[],EAST,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171231,2215814431,06/07/2022 19.58.44,PERGUN,PERSON WITH A GUN,19PZ,23XX W 95TH ST / 94XX S WESTERN AV,4264A 2262A 2262D,NaN,2300,W,95TH ST / 94XX S WESTERN,AV,[],WEST,None
171234,2215814941,06/07/2022 20.28.16,DISTME,MENTAL HEALTH DISTURBANCE,19P,54XX N BROADWAY,2023 2011,E59 452 A61,5400,N,NaN,BROADWAY,[],NORTH,None
171245,2215816273,06/07/2022 21.38.45,SHOTSF,SHOTS FIRED,19PZ,58XX N BROADWAY / 11XX W ARDMORE AV,2020,NaN,5800,N,BROADWAY / 11XX W ARDMORE,AV,[],NORTH,None
171247,2215816336,06/07/2022 21.43.04,SUPPOR,SUPPORT UNIT REQUEST,5079Z,27XX N LAKE SHORE DR,1924 1934,NaN,2700,N,LAKE SHORE,DR,[],NORTH,None


In [176]:
oemc_output.to_csv('cleaned_oemc_geocoded_final_batch.csv')

In [174]:
oemc_output.shape

(176956, 15)

In [175]:
27409/176956

0.15489161147403874

In [ ]:
#turn zipcodes into a string 
oemc_output['stripedZipCodes'] = oemc_output['cleanZipCodes'].apply(str)

In [212]:
oemc_output['stripedZipCodes'].value_counts()

None     27408
60619     8870
60620     7182
60649     6578
60617     6208
60628     5977
60626     5318
60640     5184
60644     4688
60624     4664
60629     4307
60636     4225
60651     4095
60637     3671
60621     3528
60609     3439
60623     3420
60608     3281
60612     2993
60643     2986
60625     2789
60647     2779
60618     2746
60653     2738
60639     2723
60657     2721
60641     2686
60610     2546
60632     2394
60616     2308
60615     2280
60611     1931
60607     1930
60634     1817
60614     1808
60660     1769
60622     1768
60659     1750
60605     1624
60613     1564
60630     1406
60601     1379
60652     1374
60638     1331
60645     1293
60642      815
60654      780
60602      695
60661      633
60656      612
60827      611
60707      564
60655      514
60646      483
60631      481
60604      424
60633      362
60603      309
60606      196
nan          1
Name: stripedZipCodes, dtype: int64

In [ ]:
oemc_output[oemc_output['TypeDescription'] == 'ATTEMPT SUICIDE']['cleanZipCodes'].value_counts()[0:50]

In [200]:
oemc_output.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,St_PreDir,cleanZipCodes,stripedZipCodes
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,[60620],SOUTH,60620,60620
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,[60655],SOUTH,60655,60655
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,"[60641, 60618]",WEST,60641,60641
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,[],.5,None,None
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,[60634],WEST,60634,60634


In [177]:
oemc_output['cleanZipCodes'].value_counts()

60619    8870
60649    6578
60617    6208
60620    5501
60626    5318
         ... 
60660      36
60612      31
60603      18
60643       5
60616       1
Name: cleanZipCodes, Length: 93, dtype: int64

In [178]:
oemc_output['TypeDescription'].value_counts()[0:20]

MENTAL HEALTH DISTURBANCE    59118
CHECK WELL BEING             22226
THREATENING SUICIDE          15469
DOMESTIC DISTURBANCE         12528
DISTURBANCE                  10839
EMS                           6416
ASSAULT IP                    4046
PERSON WITH A KNIFE           3187
ATTEMPT SUICIDE               2892
DOMESTIC BATTERY              2831
DISTURBANCE - MUSIC/NOISE     2475
CRIMINAL TRESPASS IP          2379
SUSPICIOUS PERSON             2343
BATTERY IP                    2272
MISSING PERSON                2093
NO POLICE SERVICE             2091
PERSON WITH A GUN             1924
SEX OFFENSE OTHER             1212
ASSAULT JO                    1088
BATTERY JO                    1056
Name: TypeDescription, dtype: int64

In [210]:
mental_disturbance = oemc_output[oemc_output['TypeDescription'] == 'MENTAL HEALTH DISTURBANCE']
mental_disturbance.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,St_PreDir,cleanZipCodes,stripedZipCodes
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,[60620],SOUTH,60620,60620
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,[],.5,None,None
8,2000100999,01/01/2020 00.52.12,DISTME,MENTAL HEALTH DISTURBANCE,5079Z,50XX S MICHIGAN AV,224R,NaN,5000,S,MICHIGAN,AV,[60615],SOUTH,60615,60615
12,2000101523,01/01/2020 01.20.34,DISTME,MENTAL HEALTH DISTURBANCE,7I,51XX N MILWAUKEE AV,1623R,NaN,5100,N,MILWAUKEE,AV,[60630],NORTH,60630,60630
15,2000101661,01/01/2020 01.28.59,DISTME,MENTAL HEALTH DISTURBANCE,19P,77XX S SAWYER AV,851R 852R,NaN,7700,S,SAWYER,AV,[60652],SOUTH,60652,60652


In [211]:
mental_disturbance['stripedZipCodes'].value_counts()

None     8944
60620    2476
60628    2171
60649    2143
60617    2049
60619    2031
60644    1850
60624    1823
60640    1683
60636    1631
60626    1622
60651    1597
60637    1358
60629    1318
60621    1233
60623    1207
60609    1163
60612    1150
60608    1067
60643    1066
60639    1006
60647     966
60618     939
60641     919
60610     901
60653     880
60625     880
60615     851
60657     778
60607     720
60632     715
60616     690
60634     620
60611     613
60622     598
60659     585
60614     580
60660     567
60652     528
60605     524
60613     488
60645     470
60638     442
60630     440
60601     414
60642     297
60654     241
60602     234
60707     226
60827     220
60661     203
60655     171
60646     162
60656     153
60631     145
60604     118
60603     107
60633      91
60606      53
nan         1
Name: stripedZipCodes, dtype: int64

In [209]:
threatening = oemc_output[oemc_output['TypeDescription'] == 'THREATENING SUICIDE']
threatening['stripedZipCodes'].value_counts()

None     2353
60640     504
60620     500
60626     459
60619     438
60649     416
60624     404
60608     376
60628     362
60641     359
60629     358
60623     356
60609     351
60617     343
60644     328
60618     320
60657     319
60647     311
60639     305
60651     290
60636     285
60611     284
60637     280
60632     279
60607     277
60612     271
60610     254
60625     252
60616     252
60621     247
60622     229
60660     202
60613     198
60634     196
60643     177
60615     176
60614     175
60605     175
60653     169
60630     164
60659     149
60638     143
60601     141
60645     120
60652     114
60642     100
60661      96
60654      83
60656      74
60631      66
60604      56
60827      51
60633      51
60655      49
60707      48
60646      47
60602      44
60603      27
60606      16
Name: stripedZipCodes, dtype: int64

In [208]:
domestic_disturbance = oemc_output[oemc_output['TypeDescription'] == 'DOMESTIC DISTURBANCE']
domestic_disturbance['stripedZipCodes'].value_counts()

60620    846
60619    829
60617    747
60649    734
None     725
60628    686
60651    489
60629    486
60624    441
60644    433
60636    422
60637    371
60643    331
60609    327
60623    317
60621    313
60647    310
60653    272
60639    221
60626    216
60632    211
60608    207
60641    194
60612    182
60618    160
60634    160
60615    151
60652    132
60625    132
60659    130
60640    129
60638    125
60616    117
60630    102
60645     98
60827     90
60622     73
60660     67
60657     65
60610     56
60613     51
60655     49
60656     47
60707     46
60614     37
60631     29
60633     25
60607     23
60611     23
60642     22
60646     19
60605     19
60601     12
60654      9
60661      9
60602      6
60603      2
60604      2
60606      1
Name: stripedZipCodes, dtype: int64

In [207]:
disturbance = oemc_output[oemc_output['TypeDescription'] == 'DISTURBANCE']
disturbance['stripedZipCodes'].value_counts()

None     1520
60619     479
60626     430
60649     417
60617     378
60640     378
60620     338
60628     306
60657     278
60636     243
60653     237
60629     232
60608     226
60621     216
60601     213
60643     211
60610     210
60611     206
60605     198
60615     193
60614     192
60625     187
60647     178
60618     177
60612     176
60624     173
60616     167
60609     161
60644     161
60607     160
60637     159
60651     152
60659     136
60613     127
60654     126
60602     121
60632     119
60630     114
60639     112
60641     109
60622      94
60660      93
60623      92
60642      80
60604      73
60603      66
60661      56
60634      52
60645      50
60638      44
60652      44
60656      41
60606      31
60646      22
60633      19
60827      18
60631      18
60655      17
60707      13
Name: stripedZipCodes, dtype: int64

In [206]:
attempt_suicide = oemc_output[oemc_output['TypeDescription'] == 'ATTEMPT SUICIDE']
attempt_suicide['stripedZipCodes'].value_counts()

None     254
60619    109
60624    101
60620     95
60644     93
60629     88
60623     87
60647     82
60649     77
60632     74
60641     74
60609     73
60618     71
60626     70
60640     69
60628     69
60651     66
60625     66
60637     66
60617     62
60608     57
60621     57
60639     55
60636     53
60643     52
60657     52
60634     48
60611     44
60660     44
60610     43
60638     42
60614     39
60613     39
60622     38
60616     37
60659     36
60630     36
60612     35
60615     33
60607     33
60653     29
60645     25
60652     22
60601     21
60642     20
60605     20
60646     17
60655     16
60707     15
60656     15
60633     14
60827     12
60631     11
60654     11
60604      9
60661      8
60602      4
60606      2
60603      2
Name: stripedZipCodes, dtype: int64

In [204]:
attempt_suicide['stripedZipCodes'].value_counts()

None     254
60619    109
60624    101
60620     95
60644     93
60629     88
60623     87
60647     82
60649     77
60632     74
60641     74
60609     73
60618     71
60626     70
60640     69
60628     69
60651     66
60625     66
60637     66
60617     62
60608     57
60621     57
60639     55
60636     53
60643     52
60657     52
60634     48
60611     44
60660     44
60610     43
60638     42
60614     39
60613     39
60622     38
60616     37
60659     36
60630     36
60612     35
60615     33
60607     33
60653     29
60645     25
60652     22
60601     21
60642     20
60605     20
60646     17
60655     16
60707     15
60656     15
60633     14
60827     12
60631     11
60654     11
60604      9
60661      8
60602      4
60606      2
60603      2
Name: stripedZipCodes, dtype: int64

In [201]:
oemc_output[oemc_output['TypeDescription'] == 'ATTEMPT SUICIDE']['stripedZipCodes'].value_counts()[0:50]

None     254
60619    109
60624    101
60620     95
60644     93
60629     88
60623     87
60647     82
60649     77
60632     74
60641     74
60609     73
60618     71
60626     70
60640     69
60628     69
60651     66
60625     66
60637     66
60617     62
60608     57
60621     57
60639     55
60636     53
60643     52
60657     52
60634     48
60611     44
60660     44
60610     43
60638     42
60614     39
60613     39
60622     38
60616     37
60659     36
60630     36
60612     35
60615     33
60607     33
60653     29
60645     25
60652     22
60601     21
60642     20
60605     20
60646     17
60655     16
60707     15
60656     15
Name: stripedZipCodes, dtype: int64

## Testing code

In [141]:
list_zipcodes0

0              [60620]
1              [60655]
2       [60641, 60618]
3                   []
4              [60634]
             ...      
5705           [60615]
5706    [60629, 60629]
5707    [60610, 60642]
5708           [60626]
5709           [60644]
Length: 5709, dtype: object

In [146]:
geoCodeToDf(0,chunkDf,list_zipcodes0)

In [147]:
chunkDf[0]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir,zipcodes
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,SOUTH,[60620]
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH,[60655]
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST,"[60641, 60618]"
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5,[]
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST,[60634]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,2003312092,02/02/2020 17.59.38,DISTME,MENTAL HEALTH DISTURBANCE,15I,11XX E 50TH ST,222,NaN,1100,E,50TH,ST,EAST,[60615]
5706,2003312108,02/02/2020 18.00.43,ASLTIP,ASSAULT IP,5B,59XX S SAWYER AV,823,NaN,5900,S,SAWYER,AV,SOUTH,"[60629, 60629]"
5707,2003312147,02/02/2020 18.03.29,DIST,DISTURBANCE,5PZ,15XX N CLYBOURN AV,1822,NaN,1500,N,CLYBOURN,AV,NORTH,"[60610, 60642]"
5708,2003312242,02/02/2020 18.11.31,CHECWB,CHECK WELL BEING,19PZ,14XX W PRATT BL,2433 2432,NaN,1400,W,PRATT,BL,WEST,[60626]


In [124]:
testChunk

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir,zipcodes
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH,[60655]
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST,"[60641, 60618]"
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5,[]
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST,[60634]
5,2000100939,01/01/2020 00.49.17,CHECWB,CHECK WELL BEING,19B,50XX N SHERIDAN RD,2041R 2024R,NaN,5000,N,SHERIDAN,RD,NORTH,[60640]
6,2000100941,01/01/2020 00.49.19,BATIP,BATTERY IP,1EZ,66XX W BELDEN AV,2512R 2510R,NaN,6600,W,BELDEN,AV,WEST,[60707]
7,2000100959,01/01/2020 00.50.02,THREAS,THREATENING SUICIDE,16I,5XX N AUSTIN BL,1511R 1512R,NaN,500,N,AUSTIN,BL,NORTH,[60644]
8,2000100999,01/01/2020 00.52.12,DISTME,MENTAL HEALTH DISTURBANCE,5079Z,50XX S MICHIGAN AV,224R,NaN,5000,S,MICHIGAN,AV,SOUTH,[60615]
9,2000101032,01/01/2020 00.53.27,DD,DOMESTIC DISTURBANCE,1IZ,10XX N DEARBORN ST,1824R 1823R,NaN,1000,N,DEARBORN,ST,NORTH,[60610]


In [123]:
test_zipcodes

1           [60655]
2    [60641, 60618]
3                []
4           [60634]
5           [60640]
6           [60707]
7           [60644]
8           [60615]
9           [60610]
dtype: object

In [117]:
test_zipcodes = testChunk.apply(lambda x: geoCodeChunk(x['numerical'],x['St_PreDir'],x['street_name'],database),axis=1)

Done
--- 0.0023156404495239258 minutes ---
Done
--- 0.0017972787221272786 minutes ---
Done
--- 0.0016391714413960775 minutes ---
Done
--- 0.0017383178075154623 minutes ---
Done
--- 0.0019673307736714682 minutes ---
Done
--- 0.0017877737681070963 minutes ---
Done
--- 0.0018693049748738607 minutes ---
Done
--- 0.0019369761149088541 minutes ---
Done
--- 0.002000558376312256 minutes ---


In [118]:
testChunk['zipcodes'] = test_zipcodes

/tmp/ipykernel_451751/4203125367.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testChunk['zipcodes'] = test_zipcodes


In [93]:
testChunk.dtypes

EventNumber          int64
EntryDate           object
EventType           object
TypeDescription     object
FinalDisposition    object
Location            object
CPDUnitList         object
CFDUnitList         object
numerical            int64
directional         object
street_name         object
suffix              object
St_PreDir           object
dtype: object

In [96]:
test_zipcodes = testChunk.apply(lambda x: geoCodeChunk(x['numerical'],x['St_PreDir'],x['street_name'],database),axis=1)

Done
--- 0.0021781166394551593 minutes ---
Done
--- 0.002099589506785075 minutes ---
Done
--- 0.0016209999720255533 minutes ---
Done
--- 0.0017386754353841147 minutes ---
Done
--- 0.001985267798105876 minutes ---
Done
--- 0.0017796754837036133 minutes ---
Done
--- 0.001896850268046061 minutes ---
Done
--- 0.0019377748171488444 minutes ---
Done
--- 0.0018540263175964356 minutes ---


In [90]:
test_zipcodes

1           [60655]
2    [60641, 60618]
3                []
4           [60634]
5           [60640]
6           [60707]
7           [60644]
8           [60615]
9           [60610]
dtype: object

In [81]:
testChunk = chunkDf[0].iloc[1:10]
testChunk.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST
5,2000100939,01/01/2020 00.49.17,CHECWB,CHECK WELL BEING,19B,50XX N SHERIDAN RD,2041R 2024R,NaN,5000,N,SHERIDAN,RD,NORTH


In [47]:
#future use case this will not work for exact address
#do not search for address bewtween 100 off
def geoCodeChunk(number,chunkDfName,database):
    list_zipcodes = []
    for index, row in chunkDfName[number].iterrows():
        complete_conditional = database[(database['St_Name'] == row['street_name']) 
                                    & (database['St_PreDir'] == row['St_PreDir'])
                                    & (database['Add_Number'] < row.numerical + 100)
                                    & (database['Add_Number'] >= row.numerical -100)
                                   ]
        list_zipcodes.append(list(set(complete_conditional['Post_Code'])))
    #print(set(complete_conditional['Post_Code']))
    print('Done')
    print("--- %s minutes ---" % ((time.time() - start_time)/60))
    return list_zipcodes
    

In [48]:
def geoCodeToCSV(number,chunkDfName,chunkZipCodeList):
    chunkDfName[number]['zipcodes']= chunkZipCodeList
    chunkDfName[number].to_csv(f'geocoded_batches/batch{number}.csv')
    

In [49]:
#creates 30 empty list 
#name convention list_zipcodes0, list_zipcodes1,..etc
for k in range(30):
    exec(f'list_zipcodes{k} = []')


In [54]:
testChunk = chunkDf[0].iloc[1:10]
testChunk.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST
5,2000100939,01/01/2020 00.49.17,CHECWB,CHECK WELL BEING,19B,50XX N SHERIDAN RD,2041R 2024R,NaN,5000,N,SHERIDAN,RD,NORTH


In [ ]:
geoCodeChunk(number,chunkDfName,database)

In [71]:
test_zipcodes = []
for index, row in testChunk.iterrows():
    complete_conditional = database[(database['St_Name'] == row['street_name']) 
                                & (database['St_PreDir'] == row['St_PreDir'])
                                & (database['Add_Number'] < row.numerical + 100)
                                & (database['Add_Number'] >= row.numerical -100)
                                ]
    test_zipcodes.append(list(set(complete_conditional['Post_Code'])))
    #print(set(complete_conditional['Post_Code']))
print('Done')
#print("--- %s minutes ---" % ((time.time() - start_time)/60))


Done


In [ ]:
from ast import literal_eval
df = pd.read_csv('test.csv', converters={'col1': literal_eval})


In [57]:
testChunk['zipcodes'] = test_zipcodes
testChunk.to_csv(f'geocoded_batches/batch_test.csv')


/tmp/ipykernel_441706/893343377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testChunk['zipcodes'] = test_zipcodes


In [59]:
testChunk.dtypes

EventNumber          int64
EntryDate           object
EventType           object
TypeDescription     object
FinalDisposition    object
Location            object
CPDUnitList         object
CFDUnitList         object
numerical            int64
directional         object
street_name         object
suffix              object
St_PreDir           object
zipcodes            object
dtype: object

In [60]:
testChunk

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir,zipcodes
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH,[60655]
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST,"[60641, 60618]"
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5,[]
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST,[60634]
5,2000100939,01/01/2020 00.49.17,CHECWB,CHECK WELL BEING,19B,50XX N SHERIDAN RD,2041R 2024R,NaN,5000,N,SHERIDAN,RD,NORTH,[60640]
6,2000100941,01/01/2020 00.49.19,BATIP,BATTERY IP,1EZ,66XX W BELDEN AV,2512R 2510R,NaN,6600,W,BELDEN,AV,WEST,[60707]
7,2000100959,01/01/2020 00.50.02,THREAS,THREATENING SUICIDE,16I,5XX N AUSTIN BL,1511R 1512R,NaN,500,N,AUSTIN,BL,NORTH,[60644]
8,2000100999,01/01/2020 00.52.12,DISTME,MENTAL HEALTH DISTURBANCE,5079Z,50XX S MICHIGAN AV,224R,NaN,5000,S,MICHIGAN,AV,SOUTH,[60615]
9,2000101032,01/01/2020 00.53.27,DD,DOMESTIC DISTURBANCE,1IZ,10XX N DEARBORN ST,1824R 1823R,NaN,1000,N,DEARBORN,ST,NORTH,[60610]


In [69]:
testChunk[['zipcodes']].iloc[0][0][0]

60655

In [70]:
testChunk[['zipcodes']].iloc[1][0][0]

60641